# Image processing to detect crops and weeds in images

### Approach 1: Using only image processing

#### Import the neccesary libraries

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import glob

%matplotlib inline
plt.rcParams['figure.figsize']
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

### For a single image

#### 1. Load the image
#### 2. Split it into red, green and blue components
#### 3. Subtract the blue and green component and just get the green component

In [ ]:
p_origin = cv2.imread(r'C:\Users\medad\OneDrive - Mahindra University\MU_ThirdYear\Sem2\DIP\Project\weed.png')
p = cv2.resize(p_origin.copy(), (400,400))

#Split the red, green, and blue components
p_blue, p_green, p_red = cv2.split(p)

#Getting the mask by subtracting red and blue componenets
p_excess_green = 128 + np.int16(p_green) - np.int16(p_blue) + np.int16(p_green) - np.int16(p_red)
p_excess_green = np.uint8(np.clip(p_excess_green, 0, 255))

mask = np.uint8((p_excess_green > 200) * 1)

plt.subplot(2, 2, 1)
plt.title('Original Image')
plt.imshow(cv2.cvtColor(p, cv2.COLOR_BGR2RGB))

plt.subplot(2, 2, 2)
plt.title('Excess green image')
plt.imshow(p_excess_green, cmap='gray')

plt.subplot(2, 2, 3)
plt.title('Mask Image')
plt.imshow(mask, cmap='gray')

plt.subplot(2, 2, 4)
plt.title('(128 + G-R + G-B) Histogram')
plt.hist(p_excess_green.ravel(), bins=256, range=[0, 256])

plt.tight_layout()
plt.show()

#### Draw contours around the masks obtained above

In [ ]:
#To draw contours
p_contoured = p.copy()

contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

cv2.drawContours(p_contoured, contours, -1, (255, 0, 0), 3)

plt.imshow(cv2.cvtColor(p_contoured, cv2.COLOR_BGR2RGB))
plt.show()

#### Draw boxes around the contours

In [ ]:
#Draw box contours 
boxes = []

for c in contours:
    x, y, w, h = cv2.boundingRect(c)
    if(w > 20 and h > 20):
        boxes.append((x, y, w, h))
        
p_temp = p.copy()

for box in boxes:
    cv2.rectangle(p_temp, (box[0], box[1]), (box[0] + box[2], box[1] + box[3]), (0, 0, 255), 4)
    
plt.imshow(p_temp)
plt.show()

print(len(boxes))
print(boxes)

### Approach 2: Using image processing and Machine Learning

#### 1. Import the necessary libraries

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from skimage import io, exposure
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

#### 2. Draw bounding boxes around the YOLO co-ordinates.

In [ ]:
#Draw bounding boxes based on the coordinates
directory = r"C:\Users\medad\OneDrive - Mahindra University\MU_ThirdYear\Sem2\DIP\Project\archive\agri_data\data"
output_folder = r"C:\Users\medad\OneDrive - Mahindra University\MU_ThirdYear\Sem2\DIP\Project\archive\agri_data\Bounding_boxes"
os.makedirs(output_folder, exist_ok=True)

#Loading images
image_files = [file for file in os.listdir(directory) if file.endswith(".jpg") or file.endswith(".jpeg")]

#Define class colors
class_colors = [(0, 255, 0), (255, 0, 0), (0, 0, 255)]  # Example: Green, Red, Blue

#Process each image
for image_file in image_files:
    image_path = os.path.join(directory, image_file)
    image = cv2.imread(image_path)
    height, width, _ = image.shape


    annotation_file = os.path.splitext(image_file)[0] + ".txt"
    annotation_path = os.path.join(directory, annotation_file)

    with open(annotation_path, "r") as file:
        annotations = file.readlines()

    #Process each annotation
    for annotation in annotations:
        class_id, x, y, w, h = map(float, annotation.split())

        # Convert the normalized coordinates to pixel values
        x_min = int((x - w/2) * width)
        y_min = int((y - h/2) * height)
        x_max = int((x + w/2) * width)
        y_max = int((y + h/2) * height)

        # Get the class color based on the class_id
        color = class_colors[int(class_id)]

        #Drawing bounding boxes
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, 2)

    #Save the image with bounding boxes
    output_path = os.path.join(output_folder, image_file)
    cv2.imwrite(output_path, image)

#     print(f"Processed: {image_file}")

print("Bounding boxes drawn and saved successfully!")

#### 3. Crop the images from the bounding boxes and put them in appropriate folders

In [ ]:
#Crop the images from the bounding boxes

directory = r"C:\Users\medad\OneDrive - Mahindra University\MU_ThirdYear\Sem2\DIP\Project\archive\agri_data\data"
output_folder = r"C:\Users\medad\OneDrive - Mahindra University\MU_ThirdYear\Sem2\DIP\Project\archive\agri_data\Cropped"
os.makedirs(output_folder, exist_ok=True)

image_files = [file for file in os.listdir(directory) if file.endswith(".jpg") or file.endswith(".jpeg")]

# Process each image
for image_file in image_files:

    image_path = os.path.join(directory, image_file)
    image = cv2.imread(image_path)
    height, width, _ = image.shape

    annotation_file = os.path.splitext(image_file)[0] + ".txt"
    annotation_path = os.path.join(directory, annotation_file)

    with open(annotation_path, "r") as file:
        annotations = file.readlines()

    # Process each annotation
    for i, annotation in enumerate(annotations):
        class_id, x, y, w, h = map(float, annotation.split())

        # Convert the normalized coordinates to pixel values
        x_min = int((x - w/2) * width)
        y_min = int((y - h/2) * height)
        x_max = int((x + w/2) * width)
        y_max = int((y + h/2) * height)

        # Crop the image based on the bounding box
        cropped_image = image[y_min:y_max, x_min:x_max]

        class_folder = os.path.join(output_folder, f"class_{int(class_id)}")
        os.makedirs(class_folder, exist_ok=True)

        output_path = os.path.join(class_folder, f"{image_file}_{i+1}.jpg")
        cv2.imwrite(output_path, cropped_image, [cv2.IMWRITE_JPEG_QUALITY, 100])

    #print(f"Processed: {image_file}")
print("Images extracted from bounding boxes and saved successfully!")

In [ ]:
#Put these images in two folders: weed and crop --> Done in the previous cell.

#### 4. Getting the appropriate paths for further preprocessing

In [ ]:
class1_folder = r"C:\Users\medad\OneDrive - Mahindra University\MU_ThirdYear\Sem2\DIP\Project\archive\agri_data\Cropped\Crop"
class2_folder = r"C:\Users\medad\OneDrive - Mahindra University\MU_ThirdYear\Sem2\DIP\Project\archive\agri_data\Cropped\Weed"

#### 5. Resize all the images(weeds & crops) to 400*400 

In [ ]:
folder_path = r"C:\Users\medad\OneDrive - Mahindra University\MU_ThirdYear\Sem2\DIP\Project\archive\agri_data\Cropped\Crop"

for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path)
        resized_image = image.resize((400, 400))
        resized_image.save(image_path)
        image.close()

In [ ]:
folder_path = r"C:\Users\medad\OneDrive - Mahindra University\MU_ThirdYear\Sem2\DIP\Project\archive\agri_data\Cropped\Weed"

for filename in os.listdir(folder_path):
    if filename.endswith('.jpg') or filename.endswith('.png'):
        image_path = os.path.join(folder_path, filename)
        image = Image.open(image_path)
        resized_image = image.resize((400, 400))
        resized_image.save(image_path)
        image.close()

#### 6. Visualising the edges using Canny edge detection

In [ ]:
# Parameters for Canny edge detection
low_threshold = 50
high_threshold = 150

# Function to extract edge features from an image and visualize the edges
def extract_and_visualize_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, low_threshold, high_threshold)
    return edges

# Load the images and extract edge features
for class_folder, class_label in [(class1_folder, 0), (class2_folder, 1)]:
    for image_file in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_file)
        image = cv2.imread(image_path)
        edges = extract_and_visualize_edges(image)

        # Plot the original image and the extracted edges
        fig, axes = plt.subplots(1, 2, figsize=(8, 4))
        axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        axes[0].set_title("Original Image")
        axes[0].axis("off")
        axes[1].imshow(edges, cmap="gray")
        axes[1].set_title("Extracted Edges")
        axes[1].axis("off")

        plt.tight_layout()

#### 7. Extracting the edges from the images and storing them with the labels in lists

In [ ]:
# Process images and extract edge features
edge_features = []
labels = []
desired_size = (400, 400) 

for class_folder, class_label in [(class1_folder, 0), (class2_folder, 1)]:
    for image_file in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_file)
        image = cv2.imread(image_path)

        resized_image = cv2.resize(image, desired_size)

        gray = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)

        # Apply Canny edge detection
        edges = cv2.Canny(gray, threshold1=30, threshold2=100)

        # Check if the number of edge features matches the expected length
        expected_length = desired_size[0] * desired_size[1]
        if len(edges.flatten()) == expected_length:
            # Append the edge features and label to the lists
            edge_features.append(edges.flatten())
            labels.append(class_label)
        else:
            # Skip the image with mismatched edge features length
            print(f"Skipping image {image_file} due to mismatched edge features length")
            print(f"Expected length: {expected_length}, Actual length: {len(edges.flatten())}")

#### 8. Convert the features to numpy arrays.

In [ ]:
# Convert the lists to numpy arrays
X = np.array(edge_features)
y = np.array(labels)

In [ ]:
print("X shape:", X.shape)
print("y shape:", y.shape)

#### 9. Split the dataset.

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### 10. Train SVM on this.

In [ ]:
# Train the Support Vector Machine (SVM) model
model = SVC()
model.fit(X_train, y_train)

#### 11. Making predictions on the test set

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

#### 12. Evaluation of the model.

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:")
print(report)

#### 13. Testing with a test image

In [ ]:
# Load the test image
test_image = cv2.imread("agri_0_136.jpeg_1.jpg")
desired_size = (400, 400)

# Resize the test image to the desired size
resized_test_image = cv2.resize(test_image, desired_size)

# Convert the test image to grayscale
gray_test = cv2.cvtColor(resized_test_image, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection to the test image
test_edges = cv2.Canny(gray_test, threshold1=30, threshold2=100)

# Reshape the test image features to match the input shape of the model
test_features = test_edges.flatten().reshape(1, -1)

# Make predictions on the test image
y_pred = model.predict(test_features)

# Get the predicted class label
predicted_class = y_pred[0]

# Map the class label to a meaningful class name (if applicable)
class_names = ["weed", "crop"]  # Define your class names
predicted_class_name = class_names[predicted_class]

# Print the predicted class label and name
print("Predicted Class Label:", predicted_class)
print("Predicted Class Name:", predicted_class_name)

# Display the test image and the prediction label using Matplotlib
fig, ax = plt.subplots(2, 1)
ax[0].imshow(cv2.cvtColor(resized_test_image, cv2.COLOR_BGR2RGB))
ax[0].set_title("Test Image")
ax[0].axis("off")
ax[1].text(0.5, 0.5, f"Predicted Class: {predicted_class_name}", fontsize=12, ha="center", va="center")
ax[1].axis("off")
plt.tight_layout()
plt.show()

### Improving the above model 

#### 1. Extracting the edges from the images using adaptive thresholding, dilation and canny edge detection

In [ ]:
# Parameters for Canny edge detection
low_threshold = 75
high_threshold = 100

# Function to extract edge features from an image and visualize the edges
def extract_and_visualize_edges(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply adaptive thresholding to obtain binary image
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Perform morphological operations to enhance the foreground regions
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    opening = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations=2)
    sure_bg = cv2.dilate(opening, kernel, iterations=3)

    # Use Canny edge detection on the foreground regions
    edges = cv2.Canny(sure_bg, low_threshold, high_threshold)
    return edges

# Absolute paths for the "Crop" and "Weed" folders
crop_folder = r"C:\Users\medad\OneDrive - Mahindra University\MU_ThirdYear\Sem2\DIP\Project\archive\agri_data\Cropped\Crop"
weed_folder = r"C:\Users\medad\OneDrive - Mahindra University\MU_ThirdYear\Sem2\DIP\Project\archive\agri_data\Cropped\Weed"
# Function to display the images and extracted edges
def display_image_with_edges(image_path):
    image = cv2.imread(image_path)
    edges = extract_and_visualize_edges(image)

    # Plot the original image and the extracted edges
    fig, axes = plt.subplots(1, 2, figsize=(8, 4))
    
    axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[0].set_title("Original Image")
    axes[0].axis("off")

    axes[1].imshow(edges, cmap="gray")
    axes[1].set_title("Extracted Edges")
    axes[1].axis("off")

    plt.tight_layout()
    plt.show()

# Print the results for the first five images in the "Crop" folder
crop_images = os.listdir(crop_folder)[:5]
for image_name in crop_images:
    image_path = os.path.join(crop_folder, image_name)
    display_image_with_edges(image_path)

# Print the results for the first five images in the "Weed" folder
weed_images = os.listdir(weed_folder)[:5]
for image_name in weed_images:
    image_path = os.path.join(weed_folder, image_name)
    display_image_with_edges(image_path)

#### 2. Extract the edge features and store them in lists

In [ ]:
# Load the images and extract edge features
edge_features = []
labels = []

for class_folder, class_label in [(crop_folder, 0), (weed_folder, 1)]:
    for image_file in os.listdir(class_folder):
        image_path = os.path.join(class_folder, image_file)
        image = cv2.imread(image_path)
        edges = extract_and_visualize_edges(image)

        # Check if the number of edge features matches the expected length
        expected_length = edges.shape[0] * edges.shape[1]
        if len(edges.flatten()) == expected_length:
            # Append the edge features and label to the lists
            edge_features.append(edges.flatten())
            labels.append(class_label)
        else:
            # Skip the image with mismatched edge features length
            print(f"Skipping image {image_file} due to mismatched edge features length")
            print(f"Expected length: {expected_length}, Actual length: {len(edges.flatten())}")

#### 3. Convert them to numpy arrays

In [ ]:
# Convert the lists to numpy arrays
edge_features = np.array(edge_features)
labels = np.array(labels)

#### 4. Display their dimensions

In [ ]:
# Print the shape of the edge features and labels arrays
print("Edge Features Shape:", edge_features.shape)
print("Labels Shape:", labels.shape)

#### 5. Split the data into train and test

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(edge_features, labels, test_size=0.2, random_state=42)

#### 6. Train the SVM model

In [ ]:
# Train an SVM classifier
svm = SVC()
svm.fit(X_train, y_train)

#### 7. Predict on the test images

In [ ]:
# Make predictions on the test set
y_pred = svm.predict(X_test)

#### 8. Evaluation of the model

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
confusion_mat = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:")
print(report)
print("Confusion Matrix:")
print(confusion_mat)

In [ ]:
# Visualize confusion matrix
plt.figure(figsize=(8, 6))
classes = ['Crop', 'Weed']
plt.imshow(confusion_mat, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np

#### 9. Test with some test images

In [ ]:
# Load the test image
test_image = cv2.imread("agri_0_136.jpeg_1.jpg")
desired_size = (400, 400)

resized_test_image = cv2.resize(test_image, desired_size)

gray_test = cv2.cvtColor(resized_test_image, cv2.COLOR_BGR2GRAY)

# Apply Canny edge detection to the test image
test_edges = cv2.Canny(gray_test, threshold1=30, threshold2=100)

test_features = test_edges.flatten().reshape(1, -1)

y_pred = svm.predict(test_features)

predicted_class = y_pred[0]

# Map the class label to a meaningful class name (if applicable)
class_names = ["Weed", "Crop"]  # Define your class names
predicted_class_name = class_names[predicted_class]

print("Predicted Class Label:", predicted_class)
print("Predicted Class Name:", predicted_class_name)

fig, ax = plt.subplots(2, 1)
ax[0].imshow(cv2.cvtColor(resized_test_image, cv2.COLOR_BGR2RGB))
ax[0].set_title("Test Image")
ax[0].axis("off")
ax[1].text(0.5, 0.5, f"Predicted Class: {predicted_class_name}", fontsize=12, ha="center", va="center")
ax[1].axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# Load the test image
test_image = cv2.imread("Weed.png")
desired_size = (400, 400)

resized_test_image = cv2.resize(test_image, desired_size)

gray_test = cv2.cvtColor(resized_test_image, cv2.COLOR_BGR2GRAY)

test_edges = cv2.Canny(gray_test, threshold1=30, threshold2=100)

# Reshape the test image features to match the input shape of the model
test_features = test_edges.flatten().reshape(1, -1)

y_pred = svm.predict(test_features)

predicted_class = y_pred[0]

# Map the class label to a meaningful class name (if applicable)
class_names = ["Weed", "Crop"]  # Define your class names
predicted_class_name = class_names[predicted_class]

print("Predicted Class Label:", predicted_class)
print("Predicted Class Name:", predicted_class_name)

fig, ax = plt.subplots(2, 1)
ax[0].imshow(cv2.cvtColor(resized_test_image, cv2.COLOR_BGR2RGB))
ax[0].set_title("Test Image")
ax[0].axis("off")
ax[1].text(0.5, 0.5, f"Predicted Class: {predicted_class_name}", fontsize=12, ha="center", va="center")
ax[1].axis("off")
plt.tight_layout()
plt.show()